##Setup

You will need to make a copy of this notebook in your Google Drive before you can edit the homework files. You can do so with **File &rarr; Save a copy in Drive**.

In [2]:
!pip install google-colab

     |████████████████████████████████| 72 kB 948 kB/s eta 0:00:01
     |████████████████████████████████| 64 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 758 kB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 47.5 MB/s eta 0:00:01
     |████████████████████████████████| 10.1 MB 27.4 MB/s eta 0:00:01eta 0:00:01     |███████████████████████         | 7.2 MB 27.4 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 9.9 MB/s s eta 0:00:01
     |████████████████████████████████| 484 kB 101.7 MB/s eta 0:00:01
     |████████████████████████████████| 118 kB 105.3 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 142.5 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 11.9 MB/s  eta 0:00:01


  Created wheel for google-colab: filename=google_colab-1.0.0-py2.py3-none-any.whl size=102289 sha256=2b7ea081cbf7508fa494c0958c0c91c98743f4a92fa81473cb523643cda2c5c3
  Stored in directory: /home/air/.cache/pip/wheels/f6/3b/58/f34ea9045a7c69bd5634978bf25ac60277e90997d9e6e74192
  Created wheel for portpicker: filename=portpicker-1.2.0-py3-none-any.whl size=13371 sha256=bfe4f1403c1cb4d05a7fcfcd36526c1a30b6e21d198228c2f66fc81f19746294
  Stored in directory: /home/air/.cache/pip/wheels/73/0c/f5/35977446e45e818e6b848be3d41e7f38298a5102f4dcda21c6
  Created wheel for tornado: filename=tornado-4.5.3-cp37-cp37m-linux_x86_64.whl size=434143 sha256=e8d0f2812401dd9c13d452e6571b753e038e5750ba33222be2203929e152889b
  Stored in directory: /home/air/.cache/pip/wheels/a2/45/43/36ec7a893e16c1212a6b1505ded0a2d73cf8e863a0227c8e04
Successfully built google-colab portpicker tornado
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninsta

In [3]:
!conda list

# packages in environment at /home/air/anaconda3/envs/elsa:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
absl-py                   0.12.0                   pypi_0    pypi
argon2-cffi               20.1.0           py37h27cfd23_1  
async_generator           1.10             py37h28b3542_0  
atari-py                  0.2.6                    pypi_0    pypi
attrs                     20.3.0                   pypi_0    pypi
automat                   20.2.0                   pypi_0    pypi
backcall                  0.2.0              pyhd3eb1b0_0  
blas                      1.0                         mkl  
bleach                    3.3.0              pyhd3eb1b0_0  
box2d-py                  2.3.8                    pypi_0    pypi
ca-certificates           2021.7.5             h06a4308_1  
cachetools                4.2.1                    pypi_0    pypi
certifi                   2021.5.30        py37h06a4308

In [14]:
#@title set up virtual display

from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

# For later
from cs285.infrastructure.colab_utils import (
    wrap_env,
    show_video
)

EasyProcessError: start error <EasyProcess cmd_param=['Xvfb', '-help'] cmd=['Xvfb', '-help'] oserror=[Errno 2] No such file or directory: 'Xvfb': 'Xvfb' return_code=None stdout="None" stderr="None" timeout_happened=False>

## Editing Code

To edit code, click the folder icon on the left menu. Navigate to the corresponding file (`cs285_f2020/...`). Double click a file to open an editor. There is a timeout of about ~12 hours with Colab while it is active (and less if you close your browser window). We sync your edits to Google Drive so that you won't lose your work in the event of an instance timeout, but you will need to re-mount your Google Drive and re-install packages with every new instance.

## Run DQN and Double DQN

In [6]:
#@title imports
import os
import time

from cs285.infrastructure.rl_trainer import RL_Trainer
from cs285.agents.dqn_agent import DQNAgent
from cs285.infrastructure.dqn_utils import get_env_kwargs

In [15]:
#@title runtime arguments

class Args:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  def __contains__(self, key):
    return hasattr(self, key)

  env_name = 'LunarLander-v3' #@param ['MsPacman-v0', 'LunarLander-v3', 'PongNoFrameSkip-v4']
  exp_name = 'q3_dqn' #@param

  ## PDF will tell you how to set ep_len
  ## and discount for each environment
  ep_len = 200 #@param {type: "integer"}

  #@markdown batches and steps
  batch_size = 32 #@param {type: "integer"}
  eval_batch_size = 1000 #@param {type: "integer"}

  num_agent_train_steps_per_iter = 1 #@param {type: "integer"}

  num_critic_updates_per_agent_update = 1 #@param {type: "integer"}
  
  #@markdown Q-learning parameters
  double_q = False #@param {type: "boolean"}

  #@markdown system
  save_params = False #@param {type: "boolean"}
  no_gpu = False #@param {type: "boolean"}
  which_gpu = 0 #@param {type: "integer"}
  seed = 1 #@param {type: "integer"}

  #@markdown logging
  ## default is to not log video so
  ## that logs are small enough to be
  ## uploaded to gradscope
  video_log_freq =  -1 #@param {type: "integer"}
  scalar_log_freq =  10000#@param {type: "integer"}


args = Args()

## ensure compatibility with hw1 code
args['train_batch_size'] = args['batch_size']

if args['video_log_freq'] > 0:
  import warnings
  warnings.warn(
      '''\nLogging videos will make eventfiles too'''
      '''\nlarge for the autograder. Set video_log_freq = -1'''
      '''\nfor the runs you intend to submit.''')

In [16]:
#@title create directories for logging

#data_path = '''/content/cs285_f2020/''' \
#        '''homework_fall2020/hw3/data'''
data_path = os.path.join(os.path.dirname(os.path.abspath('')), 'data')

if not (os.path.exists(data_path)):
    os.makedirs(data_path)

logdir = args.exp_name + '_' + args.env_name + '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

print("LOGGING TO: ", logdir)


LOGGING TO:  /home/air/workspace/cs285/cs285_dqn/cs285/data/q3_dqn_LunarLander-v3_08-07-2021_16-16-03


In [17]:
#@title Define Q-function trainer

class Q_Trainer(object):

    def __init__(self, params):
        self.params = params

        train_args = {
            'num_agent_train_steps_per_iter': params['num_agent_train_steps_per_iter'],
            'num_critic_updates_per_agent_update': params['num_critic_updates_per_agent_update'],
            'train_batch_size': params['batch_size'],
            'double_q': params['double_q'],
        }

        env_args = get_env_kwargs(params['env_name'])

        for k, v in env_args.items():
          params[k] = v

        self.params['agent_class'] = DQNAgent
        self.params['agent_params'] = params
        self.params['train_batch_size'] = params['batch_size']
        self.params['env_wrappers'] = env_args['env_wrappers']

        self.rl_trainer = RL_Trainer(self.params)

    def run_training_loop(self):
        self.rl_trainer.run_training_loop(
            self.params['num_timesteps'],
            collect_policy = self.rl_trainer.agent.actor,
            eval_policy = self.rl_trainer.agent.actor,
            )

In [18]:
#@title run training

trainer = Q_Trainer(args)
trainer.run_training_loop()

########################
logging outputs to  /home/air/workspace/cs285/cs285_dqn/cs285/data/q3_dqn_LunarLander-v3_08-07-2021_16-16-03
########################
Using GPU id 0


/home/air/anaconda3/envs/elsa/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/home/air/anaconda3/envs/elsa/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Environment '<class 'cs285.envs.box2d.lunar_lander.LunarLander'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))




********** Iteration 0 ************

Training agent...

Beginning logging procedure...
Timestep 1
mean reward (100 episodes) nan
best mean reward -inf
running time 0.006235
Train_EnvstepsSoFar : 1
TimeSinceStart : 0.006235361099243164
Done logging...




********** Iteration 1000 ************

Training agent...


********** Iteration 2000 ************

Training agent...


********** Iteration 3000 ************

Training agent...


********** Iteration 4000 ************

Training agent...


********** Iteration 5000 ************

Training agent...


********** Iteration 6000 ************

Training agent...


********** Iteration 7000 ************

Training agent...


********** Iteration 8000 ************

Training agent...


********** Iteration 9000 ************

Training agent...


********** Iteration 10000 ************

Training agent...

Beginning logging procedure...
Timestep 10001
mean reward (100 episodes) -226.384969
best mean reward -inf
running time 26.892555
Train_Envstep



********** Iteration 89000 ************

Training agent...


********** Iteration 90000 ************

Training agent...

Beginning logging procedure...
Timestep 90001
mean reward (100 episodes) -59.174972
best mean reward -59.174972
running time 282.427588
Train_EnvstepsSoFar : 90001
Train_AverageReturn : -59.174971756498906
Train_BestReturn : -59.174971756498906
TimeSinceStart : 282.4275875091553
Training Loss : 0.11266826093196869
Done logging...




********** Iteration 91000 ************

Training agent...


********** Iteration 92000 ************

Training agent...


********** Iteration 93000 ************

Training agent...


********** Iteration 94000 ************

Training agent...


********** Iteration 95000 ************

Training agent...


********** Iteration 96000 ************

Training agent...


********** Iteration 97000 ************

Training agent...


********** Iteration 98000 ************

Training agent...


********** Iteration 99000 ************

Training age



********** Iteration 174000 ************

Training agent...


********** Iteration 175000 ************

Training agent...


********** Iteration 176000 ************

Training agent...


********** Iteration 177000 ************

Training agent...


********** Iteration 178000 ************

Training agent...


********** Iteration 179000 ************

Training agent...


********** Iteration 180000 ************

Training agent...

Beginning logging procedure...
Timestep 180001
mean reward (100 episodes) 59.463186
best mean reward 59.463186
running time 564.089904
Train_EnvstepsSoFar : 180001
Train_AverageReturn : 59.46318571135001
Train_BestReturn : 59.46318571135001
TimeSinceStart : 564.0899040699005
Training Loss : 0.23810216784477234
Done logging...




********** Iteration 181000 ************

Training agent...


********** Iteration 182000 ************

Training agent...


********** Iteration 183000 ************

Training agent...


********** Iteration 184000 ************

Train



********** Iteration 261000 ************

Training agent...


********** Iteration 262000 ************

Training agent...


********** Iteration 263000 ************

Training agent...


********** Iteration 264000 ************

Training agent...


********** Iteration 265000 ************

Training agent...


********** Iteration 266000 ************

Training agent...


********** Iteration 267000 ************

Training agent...


********** Iteration 268000 ************

Training agent...


********** Iteration 269000 ************

Training agent...


********** Iteration 270000 ************

Training agent...

Beginning logging procedure...
Timestep 270001
mean reward (100 episodes) 69.440198
best mean reward 87.531794
running time 823.158648
Train_EnvstepsSoFar : 270001
Train_AverageReturn : 69.4401980314906
Train_BestReturn : 87.53179441135585
TimeSinceStart : 823.15864777565
Training Loss : 0.12258537858724594
Done logging...




********** Iteration 271000 ************

Training

Train_EnvstepsSoFar : 350001
Train_AverageReturn : 21.296638442466264
Train_BestReturn : 89.25235409306353
TimeSinceStart : 1045.4496676921844
Training Loss : 0.16915282607078552
Done logging...




********** Iteration 351000 ************

Training agent...


********** Iteration 352000 ************

Training agent...


********** Iteration 353000 ************

Training agent...


********** Iteration 354000 ************

Training agent...


********** Iteration 355000 ************

Training agent...


********** Iteration 356000 ************

Training agent...


********** Iteration 357000 ************

Training agent...


********** Iteration 358000 ************

Training agent...


********** Iteration 359000 ************

Training agent...


********** Iteration 360000 ************

Training agent...

Beginning logging procedure...
Timestep 360001
mean reward (100 episodes) 1.902393
best mean reward 89.252354
running time 1071.445028
Train_EnvstepsSoFar : 360001
Train_AverageRetur



********** Iteration 437000 ************

Training agent...


********** Iteration 438000 ************

Training agent...


********** Iteration 439000 ************

Training agent...


********** Iteration 440000 ************

Training agent...

Beginning logging procedure...
Timestep 440001
mean reward (100 episodes) 96.152777
best mean reward 108.076698
running time 1291.139858
Train_EnvstepsSoFar : 440001
Train_AverageReturn : 96.15277723494162
Train_BestReturn : 108.07669766345708
TimeSinceStart : 1291.139858007431
Training Loss : 0.271650493144989
Done logging...




********** Iteration 441000 ************

Training agent...


********** Iteration 442000 ************

Training agent...


********** Iteration 443000 ************

Training agent...


********** Iteration 444000 ************

Training agent...


********** Iteration 445000 ************

Training agent...


********** Iteration 446000 ************

Training agent...


********** Iteration 447000 ************

Trai

In [ ]:
#@markdown You can visualize your runs with tensorboard from within the notebook

## requires tensorflow==2.3.0
%load_ext tensorboard
%tensorboard --logdir /content/cs285_f2020/homework_fall2020/hw3/data/